In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
from tqdm import tqdm 
from datetime import datetime

pd.options.display.max_columns = 200
plt.style.use('seaborn-whitegrid')

## Read Exchange Rate Data From `IMF`.org 
- Step 1. Read sample data from IMF website using `pd.read_html`.
- Step 2. Extract data from response. 
- Step 3. Create a dataframe that includes date range for the analysis - `2010-01-01`. 
- Step 4. Clean Up Columns using `regex`
- Step 5. Adjust outlier currencies using `apply()`
- Step 6. Observe/explore null values.
- Step 7. Save the final data to a csv file. 

### Step 1. Read sample data from IMP website using `pd.read_html`

In [2]:
select_date = '2010-01-01'
url = f'https://www.imf.org/external/np/fin/data/rms_mth.aspx?SelectDate={select_date}&reportType=REP'
response_df = pd.read_html(url)
response_df


[                                                   0  \
 0  Representative Exchange Rates for Selected Cur...   
 
                                    1  
 0  Download this fileTSV -- TSV tips  ,
                                    0
 0  Download this fileTSV -- TSV tips,
   Download this file
 0    TSV -- TSV tips,
                                                    0
 0  These rates, normally quoted as currency units...
 1  Representative Exchange Rates for Selected Cur...
 2  Representative Exchange Rates for Selected Cur...,
                                                    0
 0  These rates, normally quoted as currency units...,
    Representative Exchange Rates for Selected Currencies for January 2010  \
                                                                  Currency   
 0                                        Chinese yuan                       
 1                                           Euro  (1)                       
 2                                        J

### Extract data from response.

In [3]:
# The first half of the exchange rate data (table 1) appears on the fifth column
response_df[5]

Representative Exchange Rates for Selected Currencies for January 2010  \
                                                                 Currency   
0                                        Chinese yuan                       
1                                           Euro  (1)                       
2                                        Japanese yen                       
3                                     U.K. pound  (1)                       
4                                         U.S. dollar                       
5                                      Algerian dinar                       
6                              Australian dollar  (1)                       
7                                       Bahrain dinar                       
8                                  Botswana pula  (1)                       
9                                      Brazilian real                       
10                                      Brunei dollar                       
11                                    Canadian dollar                       
12                                       Chilean peso                       
13                                     Colombian peso                       
14                                       Czech koruna                       
15                                       Danish krone                       
16                                   Hungarian forint                       
17                                    Icelandic krona                       
18                                       Indian rupee                       
19                                  Indonesian rupiah                       
20                                       Iranian rial                       
21                                 Israeli New Shekel                       
22                                  Kazakhstani tenge                       
23                                         Korean won                       
24                                      Kuwaiti dinar                       
25                                  Libyan dinar  (2)                       
26                                  Malaysian ringgit                       
27                                    Mauritian rupee                       
28                                       Mexican peso                       
29                                     Nepalese rupee                       
30                            New Zealand dollar  (1)                       
31                                    Norwegian krone                       
32                                         Omani rial                       
33                                    Pakistani rupee                       
34                                       Peruvian sol                       
35                                    Philippine peso                       
36                                       Polish zloty                       
37                                       Qatari riyal                       
38                                      Russian ruble                       
39                                Saudi Arabian riyal                       
40                                   Singapore dollar                       
41                                 South African rand                       
42                                   Sri Lankan rupee                       
43                                      Swedish krona                       
44                                        Swiss franc                       
45                                          Thai baht                       
46                                 Trinidadian dollar                       
47                                     Tunisian dinar                       
48                                      U.A.E. dirham                       
49                                     Uruguayan peso                       

In [4]:
# and the second half in the response_df[6] column
response_df[6]

Representative Exchange Rates for Selected Currencies for January 2010 (Continued)  \
                                                                             Currency   
0                                        Chinese yuan                                   
1                                           Euro  (1)                                   
2                                        Japanese yen                                   
3                                     U.K. pound  (1)                                   
4                                         U.S. dollar                                   
5                                      Algerian dinar                                   
6                              Australian dollar  (1)                                   
7                                       Bahrain dinar                                   
8                                  Botswana pula  (1)                                   
9                                      Brazilian real                                   
10                                      Brunei dollar                                   
11                                    Canadian dollar                                   
12                                       Chilean peso                                   
13                                     Colombian peso                                   
14                                       Czech koruna                                   
15                                       Danish krone                                   
16                                   Hungarian forint                                   
17                                    Icelandic krona                                   
18                                       Indian rupee                                   
19                                  Indonesian rupiah                                   
20                                       Iranian rial                                   
21                                 Israeli New Shekel                                   
22                                  Kazakhstani tenge                                   
23                                         Korean won                                   
24                                      Kuwaiti dinar                                   
25                                  Libyan dinar  (2)                                   
26                                  Malaysian ringgit                                   
27                                    Mauritian rupee                                   
28                                       Mexican peso                                   
29                                     Nepalese rupee                                   
30                            New Zealand dollar  (1)                                   
31                                    Norwegian krone                                   
32                                         Omani rial                                   
33                                    Pakistani rupee                                   
34                                       Peruvian sol                                   
35                                    Philippine peso                                   
36                                       Polish zloty                                   
37                                       Qatari riyal                                   
38                                      Russian ruble                                   
39                                Saudi Arabian riyal                                   
40                                   Singapore dollar                                   
41                                 South African rand                                   
42                                   Sri Lankan rupee                                  

- Inspect the first table - response_df[5]

In [5]:
# inspect the columns
response_df[5].columns

MultiIndex([('Representative Exchange Rates for Selected Currencies for January 2010', ...),
            ('Representative Exchange Rates for Selected Currencies for January 2010', ...),
            ('Representative Exchange Rates for Selected Currencies for January 2010', ...),
            ('Representative Exchange Rates for Selected Currencies for January 2010', ...),
            ('Representative Exchange Rates for Selected Currencies for January 2010', ...),
            ('Representative Exchange Rates for Selected Currencies for January 2010', ...),
            ('Representative Exchange Rates for Selected Currencies for January 2010', ...),
            ('Representative Exchange Rates for Selected Currencies for January 2010', ...),
            ('Representative Exchange Rates for Selected Currencies for January 2010', ...),
            ('Representative Exchange Rates for Selected Currencies for January 2010', ...),
            ('Representative Exchange Rates for Selected Currencies fo

In [6]:
# drop the first level of the multiindex 
response_df[5].columns.droplevel(0)

Index(['Currency', 'January 04, 2010', 'January 05, 2010', 'January 06, 2010',
       'January 07, 2010', 'January 08, 2010', 'January 11, 2010',
       'January 12, 2010', 'January 13, 2010', 'January 14, 2010',
       'January 15, 2010', 'January 18, 2010'],
      dtype='object')

In [7]:
# overwrite the df
response_df[5].columns = response_df[5].columns.droplevel(0)

In [8]:
# set curency as index and flip the rows with columns
response_df[5].set_index('Currency').T

Currency,Chinese yuan,Euro (1),Japanese yen,U.K. pound (1),U.S. dollar,Algerian dinar,Australian dollar (1),Bahrain dinar,Botswana pula (1),Brazilian real,Brunei dollar,Canadian dollar,Chilean peso,Colombian peso,Czech koruna,Danish krone,Hungarian forint,Icelandic krona,Indian rupee,Indonesian rupiah,Iranian rial,Israeli New Shekel,Kazakhstani tenge,Korean won,Kuwaiti dinar,Libyan dinar (2),Malaysian ringgit,Mauritian rupee,Mexican peso,Nepalese rupee,New Zealand dollar (1),Norwegian krone,Omani rial,Pakistani rupee,Peruvian sol,Philippine peso,Polish zloty,Qatari riyal,Russian ruble,Saudi Arabian riyal,Singapore dollar,South African rand,Sri Lankan rupee,Swedish krona,Swiss franc,Thai baht,Trinidadian dollar,Tunisian dinar,U.A.E. dirham,Uruguayan peso,Bolivar Fuerte
"January 04, 2010",6.8281,1.4389,93.05,1.6223,1.0,NaN,0.8970,0.376,0.1506,1.7404,1.4038,1.0378,507.10,2044.23,18.275,5.1717,187.23,124.85,46.51,9330.0,10009.0,3.765,148.46,1167.6,0.28710,1.932367,3.4195,30.3606,12.9226,74.60,NaN,5.7190,0.3845,84.4322,NaN,NaN,2.8465,3.64,NaN,3.75,1.4038,7.315,114.4253,7.1425,1.0336,33.359,6.3574,NaN,3.6725,NaN,2.1446
"January 05, 2010",6.8278,1.4442,91.75,1.6014,1.0,NaN,0.9133,0.376,0.1514,1.7232,1.3969,1.0372,505.70,2021.21,18.164,5.1527,186.09,124.78,46.20,9308.0,9980.0,3.736,148.46,1156.2,0.28660,1.932367,3.3885,30.3474,12.8241,74.11,0.7344,5.6862,0.3845,84.6402,NaN,NaN,2.8264,3.64,NaN,3.75,1.3969,7.285,114.4404,7.0575,1.0307,33.278,6.3548,NaN,3.6725,NaN,2.1446
"January 06, 2010",6.8277,1.4350,91.60,1.5984,1.0,NaN,0.9138,0.376,0.1504,1.7219,1.3966,1.0334,504.22,1992.78,18.352,5.1852,187.30,125.29,46.12,9308.0,10000.0,3.740,148.40,1144.3,0.28690,1.932367,3.3875,30.3042,12.7489,73.88,0.7343,5.7059,0.3845,84.7199,NaN,NaN,2.8493,3.64,NaN,3.75,1.3966,7.335,114.3429,NaN,1.0335,33.156,6.3640,NaN,3.6725,NaN,2.1446
"January 07, 2010",6.8276,1.4304,92.20,1.5912,1.0,NaN,0.9225,0.376,0.1494,1.7329,1.3942,1.0351,497.93,1971.32,18.436,5.2021,187.65,125.54,45.87,9228.0,9981.0,3.727,148.22,1138.2,0.28670,1.932367,3.3705,30.2026,12.7646,73.60,0.7378,5.7313,0.3845,84.7822,NaN,NaN,2.8631,3.64,NaN,3.75,1.3942,7.395,114.3298,7.1125,1.0350,33.160,6.3610,NaN,3.6725,NaN,2.1446
"January 08, 2010",6.8279,1.4273,93.30,1.6007,1.0,NaN,0.9154,0.376,0.1490,1.7405,1.3982,1.0344,494.54,1969.08,18.436,5.2133,187.91,126.21,45.80,9240.0,NaN,3.718,148.22,1133.2,0.28670,1.932367,3.3785,30.1882,12.7243,73.01,0.7325,5.7237,0.3845,84.7547,NaN,NaN,2.8683,3.64,NaN,3.75,1.3982,7.445,114.3221,7.1225,1.0364,33.130,6.3343,NaN,3.6725,NaN,2.1446
"January 11, 2010",6.8275,1.4528,NaN,1.6176,1.0,NaN,0.9309,0.376,0.1511,1.7350,1.3880,1.0317,492.37,1968.24,18.034,5.1219,184.00,124.28,45.36,9130.0,9959.0,3.683,148.17,1132.9,0.28630,1.932367,3.3335,30.0681,12.6692,73.20,0.7395,5.6026,0.3845,84.7904,NaN,NaN,2.7983,3.64,29.4283,3.75,1.3880,7.310,114.3944,7.0325,1.0164,33.160,6.3116,NaN,3.6725,NaN,2.5935
"January 12, 2010",6.8274,1.4481,92.25,1.6141,1.0,NaN,0.9272,0.376,0.1492,1.7315,1.3910,1.0375,489.47,1968.24,18.131,5.1381,184.65,124.09,45.40,9185.0,9964.0,3.691,148.15,1119.8,0.28640,1.932367,3.3475,29.9202,12.7291,72.60,0.7422,5.6505,0.3845,84.8411,NaN,NaN,2.8167,3.64,29.3774,3.75,1.3910,7.475,114.3394,7.0350,1.0190,33.059,6.3318,NaN,3.6725,NaN,2.5935
"January 13, 2010",6.8273,1.4563,91.12,1.6277,1.0,NaN,0.9238,0.376,0.1504,1.7431,1.3897,1.0322,491.16,1959.43,17.965,5.1092,184.17,124.19,45.68,9180.0,9964.0,3.667,148.11,1124.2,0.28620,1.932367,3.3495,29.8075,12.7698,72.85,0.7385,5.6159,0.3845,84.8130,NaN,NaN,2.8009,3.64,29.6409,3.75,1.3897,7.365,114.3190,7.0600,1.0163,33.060,6.3619,NaN,3.6725,NaN,2.5935
"January 14, 2010",6.8272,1.4486,91.75,1.6270,1.0,NaN,0.9296,0.376,0.1495,1.7434,1.3864,1.0261,490.01,1965.81,17.978,5.1362,184.10,124.22,45.48,9150.0,9961.0,3.683,148.08,1126.4,0.28605,1.932367,3.3330,29.7037,12.7364,72.90,0.7398,5.6358,0.3845,84.7848,NaN,NaN,2.7930,3.64,29.4299,3.75,1.3864,7.440,NaN,7.0200,1.0213,33.051,6.3547,NaN,3.6725,NaN,2.5935
"January 15, 2010",6.8271,1.4374,

In [9]:
# save the results to a seperate temporary dataframe 
temp_df = response_df[5].set_index('Currency').T

- Change the `Currency` index column type from str to datetime

In [10]:
temp_df.index

Index(['January 04, 2010', 'January 05, 2010', 'January 06, 2010',
       'January 07, 2010', 'January 08, 2010', 'January 11, 2010',
       'January 12, 2010', 'January 13, 2010', 'January 14, 2010',
       'January 15, 2010', 'January 18, 2010'],
      dtype='object')

In [11]:
pd.to_datetime(temp_df.index.str.replace(',', ', '))

DatetimeIndex(['2010-01-04', '2010-01-05', '2010-01-06', '2010-01-07',
               '2010-01-08', '2010-01-11', '2010-01-12', '2010-01-13',
               '2010-01-14', '2010-01-15', '2010-01-18'],
              dtype='datetime64[ns]', freq=None)

In [12]:
# update temp_df
temp_df.index = pd.to_datetime(temp_df.index.str.replace(',', ', '))

- Apply the same logic for the second table - response_df[6]

In [13]:
response_df[6].columns = response_df[6].columns.droplevel(0)
temp_df2 = response_df[6].set_index('Currency').T
temp_df2.index = pd.to_datetime(temp_df2.index.str.replace(',', ', '))

- Join both temporary dataframes

In [14]:
joined_df = pd.concat([temp_df, temp_df2])
joined_df


Currency,Chinese yuan,Euro (1),Japanese yen,U.K. pound (1),U.S. dollar,Algerian dinar,Australian dollar (1),Bahrain dinar,Botswana pula (1),Brazilian real,Brunei dollar,Canadian dollar,Chilean peso,Colombian peso,Czech koruna,Danish krone,Hungarian forint,Icelandic krona,Indian rupee,Indonesian rupiah,Iranian rial,Israeli New Shekel,Kazakhstani tenge,Korean won,Kuwaiti dinar,Libyan dinar (2),Malaysian ringgit,Mauritian rupee,Mexican peso,Nepalese rupee,New Zealand dollar (1),Norwegian krone,Omani rial,Pakistani rupee,Peruvian sol,Philippine peso,Polish zloty,Qatari riyal,Russian ruble,Saudi Arabian riyal,Singapore dollar,South African rand,Sri Lankan rupee,Swedish krona,Swiss franc,Thai baht,Trinidadian dollar,Tunisian dinar,U.A.E. dirham,Uruguayan peso,Bolivar Fuerte
2010-01-04,6.8281,1.4389,93.05,1.6223,1.0,NaN,0.8970,0.376,0.1506,1.7404,1.4038,1.0378,507.10,2044.23,18.275,5.1717,187.23,124.85,46.51,9330.0,10009.0,3.765,148.46,1167.6,0.28710,1.932367,3.4195,30.3606,12.9226,74.60,NaN,5.7190,0.3845,84.4322,NaN,NaN,2.8465,3.64,NaN,3.75,1.4038,7.315,114.4253,7.1425,1.0336,33.359,6.3574,NaN,3.6725,NaN,2.1446
2010-01-05,6.8278,1.4442,91.75,1.6014,1.0,NaN,0.9133,0.376,0.1514,1.7232,1.3969,1.0372,505.70,2021.21,18.164,5.1527,186.09,124.78,46.20,9308.0,9980.0,3.736,148.46,1156.2,0.28660,1.932367,3.3885,30.3474,12.8241,74.11,0.7344,5.6862,0.3845,84.6402,NaN,NaN,2.8264,3.64,NaN,3.75,1.3969,7.285,114.4404,7.0575,1.0307,33.278,6.3548,NaN,3.6725,NaN,2.1446
2010-01-06,6.8277,1.4350,91.60,1.5984,1.0,NaN,0.9138,0.376,0.1504,1.7219,1.3966,1.0334,504.22,1992.78,18.352,5.1852,187.30,125.29,46.12,9308.0,10000.0,3.740,148.40,1144.3,0.28690,1.932367,3.3875,30.3042,12.7489,73.88,0.7343,5.7059,0.3845,84.7199,NaN,NaN,2.8493,3.64,NaN,3.75,1.3966,7.335,114.3429,NaN,1.0335,33.156,6.3640,NaN,3.6725,NaN,2.1446
2010-01-07,6.8276,1.4304,92.20,1.5912,1.0,NaN,0.9225,0.376,0.1494,1.7329,1.3942,1.0351,497.93,1971.32,18.436,5.2021,187.65,125.54,45.87,9228.0,9981.0,3.727,148.22,1138.2,0.28670,1.932367,3.3705,30.2026,12.7646,73.60,0.7378,5.7313,0.3845,84.7822,NaN,NaN,2.8631,3.64,NaN,3.75,1.3942,7.395,114.3298,7.1125,1.0350,33.160,6.3610,NaN,3.6725,NaN,2.1446
2010-01-08,6.8279,1.4273,93.30,1.6007,1.0,NaN,0.9154,0.376,0.1490,1.7405,1.3982,1.0344,494.54,1969.08,18.436,5.2133,187.91,126.21,45.80,9240.0,NaN,3.718,148.22,1133.2,0.28670,1.932367,3.3785,30.1882,12.7243,73.01,0.7325,5.7237,0.3845,84.7547,NaN,NaN,2.8683,3.64,NaN,3.75,1.3982,7.445,114.3221,7.1225,1.0364,33.130,6.3343,NaN,3.6725,NaN,2.1446
2010-01-11,6.8275,1.4528,NaN,1.6176,1.0,NaN,0.9309,0.376,0.1511,1.7350,1.3880,1.0317,492.37,1968.24,18.034,5.1219,184.00,124.28,45.36,9130.0,9959.0,3.683,148.17,1132.9,0.28630,1.932367,3.3335,30.0681,12.6692,73.20,0.7395,5.6026,0.3845,84.7904,NaN,NaN,2.7983,3.64,29.4283,3.75,1.3880,7.310,114.3944,7.0325,1.0164,33.160,6.3116,NaN,3.6725,NaN,2.5935
2010-01-12,6.8274,1.4481,92.25,1.6141,1.0,NaN,0.9272,0.376,0.1492,1.7315,1.3910,1.0375,489.47,1968.24,18.131,5.1381,184.65,124.09,45.40,9185.0,9964.0,3.691,148.15,1119.8,0.28640,1.932367,3.3475,29.9202,12.7291,72.60,0.7422,5.6505,0.3845,84.8411,NaN,NaN,2.8167,3.64,29.3774,3.75,1.3910,7.475,114.3394,7.0350,1.0190,33.059,6.3318,NaN,3.6725,NaN,2.5935
2010-01-13,6.8273,1.4563,91.12,1.6277,1.0,NaN,0.9238,0.376,0.1504,1.7431,1.3897,1.0322,491.16,1959.43,17.965,5.1092,184.17,124.19,45.68,9180.0,9964.0,3.667,148.11,1124.2,0.28620,1.932367,3.3495,29.8075,12.7698,72.85,0.7385,5.6159,0.3845,84.8130,NaN,NaN,2.8009,3.64,29.6409,3.75,1.3897,7.365,114.3190,7.0600,1.0163,33.060,6.3619,NaN,3.6725,NaN,2.5935
2010-01-14,6.8272,1.4486,91.75,1.6270,1.0,NaN,0.9296,0.376,0.1495,1.7434,1.3864,1.0261,490.01,1965.81,17.978,5.1362,184.10,124.22,45.48,9150.0,9961.0,3.683,148.08,1126.4,0.28605,1.932367,3.3330,29.7037,12.7364,72.90,0.7398,5.6358,0.3845,84.7848,NaN,NaN,2.7930,3.64,29.4299,3.75,1.3864,7.440,NaN,7.0200,1.0213,33.051,6.3547,NaN,3.6725,NaN,2.5935
2010-01-15,6.8271,1.4374,91.10,1.6288,1.0,NaN,0.9272,0.376,0.1497,1.7640,1.3886,1.0287,493.00,1974.13,18.

- Step 3. Create a dataframe that includes date range for the analysis - `2010-01-01`. 

In [15]:
df = pd.DataFrame(pd.date_range('2010-01-01', datetime.today()), columns=['dates'])
df.head()

,dates
0,2010-01-01
1,2010-01-02
2,2010-01-03
3,2010-01-04
4,2010-01-05


In [16]:
# add year and moth columns
df['year'] = df['dates'].dt.year
df['month'] = df['dates'].dt.month 

In [17]:
df.head()

,dates,year,month
0,2010-01-01,2010,1
1,2010-01-02,2010,1
2,2010-01-03,2010,1
3,2010-01-04,2010,1
4,2010-01-05,2010,1


In [18]:
# group the dates based on year and month
df.groupby(['year', 'month'])['dates'].min()

year  month
2010  1       2010-01-01
      2       2010-02-01
      3       2010-03-01
      4       2010-04-01
      5       2010-05-01
                 ...    
2022  10      2022-10-01
      11      2022-11-01
      12      2022-12-01
2023  1       2023-01-01
      2       2023-02-01
Name: dates, Length: 158, dtype: datetime64[ns]

In [19]:
# reset the index to default and overwrite df
df.groupby(['year', 'month'])['dates'].min().index

MultiIndex([(2010,  1),
            (2010,  2),
            (2010,  3),
            (2010,  4),
            (2010,  5),
            (2010,  6),
            (2010,  7),
            (2010,  8),
            (2010,  9),
            (2010, 10),
            ...
            (2022,  5),
            (2022,  6),
            (2022,  7),
            (2022,  8),
            (2022,  9),
            (2022, 10),
            (2022, 11),
            (2022, 12),
            (2023,  1),
            (2023,  2)],
           names=['year', 'month'], length=158)

In [20]:
df = df.groupby(['year', 'month'])['dates'].min().reset_index()
df.head()

,year,month,dates
0,2010,1,2010-01-01
1,2010,2,2010-02-01
2,2010,3,2010-03-01
3,2010,4,2010-04-01
4,2010,5,2010-05-01


In [21]:
dfs = []

for date in tqdm(df['dates']):
    try:
        url = f'https://www.imf.org/external/np/fin/data/rms_mth.aspx?SelectDate={date:%Y-%m-%d}&reportType=REP'
        response_df = pd.read_html(url)
        # table 1
        response_df[5].columns = response_df[5].columns.droplevel(0)
        temp_df = response_df[5].set_index('Currency').T
        temp_df.index = pd.to_datetime(temp_df.index.astype('str').str.replace(',', ', '))
        # when there is no table 2 due to date, skip table 2
        if response_df[6].shape == (2, 2):
            dfs.append(temp_df)
            continue
        # table 2
        response_df[6].columns = response_df[6].columns.droplevel(0)
        temp_df2 = response_df[6].set_index('Currency').T
        temp_df2.index = pd.to_datetime(temp_df2.index.astype('str').str.replace(',', ', '))
        # concatenate tables
        combined_df = pd.concat([temp_df, temp_df2])
        # add the df to dfs
        dfs.append(combined_df)
    except Exception as e:
        print(f'Exception for {date:%Y-%m-%d} : {e}')



100%|██████████| 158/158 [06:23<00:00,  2.43s/it]


In [22]:
# check a sample df
dfs[0]

Currency,Chinese yuan,Euro (1),Japanese yen,U.K. pound (1),U.S. dollar,Algerian dinar,Australian dollar (1),Bahrain dinar,Botswana pula (1),Brazilian real,Brunei dollar,Canadian dollar,Chilean peso,Colombian peso,Czech koruna,Danish krone,Hungarian forint,Icelandic krona,Indian rupee,Indonesian rupiah,Iranian rial,Israeli New Shekel,Kazakhstani tenge,Korean won,Kuwaiti dinar,Libyan dinar (2),Malaysian ringgit,Mauritian rupee,Mexican peso,Nepalese rupee,New Zealand dollar (1),Norwegian krone,Omani rial,Pakistani rupee,Peruvian sol,Philippine peso,Polish zloty,Qatari riyal,Russian ruble,Saudi Arabian riyal,Singapore dollar,South African rand,Sri Lankan rupee,Swedish krona,Swiss franc,Thai baht,Trinidadian dollar,Tunisian dinar,U.A.E. dirham,Uruguayan peso,Bolivar Fuerte
2010-01-04,6.8281,1.4389,93.05,1.6223,1.0,NaN,0.8970,0.376,0.1506,1.7404,1.4038,1.0378,507.10,2044.23,18.275,5.1717,187.23,124.85,46.51,9330.0,10009.0,3.765,148.46,1167.6,0.28710,1.932367,3.4195,30.3606,12.9226,74.60,NaN,5.7190,0.3845,84.4322,NaN,NaN,2.8465,3.64,NaN,3.75,1.4038,7.315,114.4253,7.1425,1.0336,33.359,6.3574,NaN,3.6725,NaN,2.1446
2010-01-05,6.8278,1.4442,91.75,1.6014,1.0,NaN,0.9133,0.376,0.1514,1.7232,1.3969,1.0372,505.70,2021.21,18.164,5.1527,186.09,124.78,46.20,9308.0,9980.0,3.736,148.46,1156.2,0.28660,1.932367,3.3885,30.3474,12.8241,74.11,0.7344,5.6862,0.3845,84.6402,NaN,NaN,2.8264,3.64,NaN,3.75,1.3969,7.285,114.4404,7.0575,1.0307,33.278,6.3548,NaN,3.6725,NaN,2.1446
2010-01-06,6.8277,1.4350,91.60,1.5984,1.0,NaN,0.9138,0.376,0.1504,1.7219,1.3966,1.0334,504.22,1992.78,18.352,5.1852,187.30,125.29,46.12,9308.0,10000.0,3.740,148.40,1144.3,0.28690,1.932367,3.3875,30.3042,12.7489,73.88,0.7343,5.7059,0.3845,84.7199,NaN,NaN,2.8493,3.64,NaN,3.75,1.3966,7.335,114.3429,NaN,1.0335,33.156,6.3640,NaN,3.6725,NaN,2.1446
2010-01-07,6.8276,1.4304,92.20,1.5912,1.0,NaN,0.9225,0.376,0.1494,1.7329,1.3942,1.0351,497.93,1971.32,18.436,5.2021,187.65,125.54,45.87,9228.0,9981.0,3.727,148.22,1138.2,0.28670,1.932367,3.3705,30.2026,12.7646,73.60,0.7378,5.7313,0.3845,84.7822,NaN,NaN,2.8631,3.64,NaN,3.75,1.3942,7.395,114.3298,7.1125,1.0350,33.160,6.3610,NaN,3.6725,NaN,2.1446
2010-01-08,6.8279,1.4273,93.30,1.6007,1.0,NaN,0.9154,0.376,0.1490,1.7405,1.3982,1.0344,494.54,1969.08,18.436,5.2133,187.91,126.21,45.80,9240.0,NaN,3.718,148.22,1133.2,0.28670,1.932367,3.3785,30.1882,12.7243,73.01,0.7325,5.7237,0.3845,84.7547,NaN,NaN,2.8683,3.64,NaN,3.75,1.3982,7.445,114.3221,7.1225,1.0364,33.130,6.3343,NaN,3.6725,NaN,2.1446
2010-01-11,6.8275,1.4528,NaN,1.6176,1.0,NaN,0.9309,0.376,0.1511,1.7350,1.3880,1.0317,492.37,1968.24,18.034,5.1219,184.00,124.28,45.36,9130.0,9959.0,3.683,148.17,1132.9,0.28630,1.932367,3.3335,30.0681,12.6692,73.20,0.7395,5.6026,0.3845,84.7904,NaN,NaN,2.7983,3.64,29.4283,3.75,1.3880,7.310,114.3944,7.0325,1.0164,33.160,6.3116,NaN,3.6725,NaN,2.5935
2010-01-12,6.8274,1.4481,92.25,1.6141,1.0,NaN,0.9272,0.376,0.1492,1.7315,1.3910,1.0375,489.47,1968.24,18.131,5.1381,184.65,124.09,45.40,9185.0,9964.0,3.691,148.15,1119.8,0.28640,1.932367,3.3475,29.9202,12.7291,72.60,0.7422,5.6505,0.3845,84.8411,NaN,NaN,2.8167,3.64,29.3774,3.75,1.3910,7.475,114.3394,7.0350,1.0190,33.059,6.3318,NaN,3.6725,NaN,2.5935
2010-01-13,6.8273,1.4563,91.12,1.6277,1.0,NaN,0.9238,0.376,0.1504,1.7431,1.3897,1.0322,491.16,1959.43,17.965,5.1092,184.17,124.19,45.68,9180.0,9964.0,3.667,148.11,1124.2,0.28620,1.932367,3.3495,29.8075,12.7698,72.85,0.7385,5.6159,0.3845,84.8130,NaN,NaN,2.8009,3.64,29.6409,3.75,1.3897,7.365,114.3190,7.0600,1.0163,33.060,6.3619,NaN,3.6725,NaN,2.5935
2010-01-14,6.8272,1.4486,91.75,1.6270,1.0,NaN,0.9296,0.376,0.1495,1.7434,1.3864,1.0261,490.01,1965.81,17.978,5.1362,184.10,124.22,45.48,9150.0,9961.0,3.683,148.08,1126.4,0.28605,1.932367,3.3330,29.7037,12.7364,72.90,0.7398,5.6358,0.3845,84.7848,NaN,NaN,2.7930,3.64,29.4299,3.75,1.3864,7.440,NaN,7.0200,1.0213,33.051,6.3547,NaN,3.6725,NaN,2.5935
2010-01-15,6.8271,1.4374,91.10,1.6288,1.0,NaN,0.9272,0.376,0.1497,1.7640,1.3886,1.0287,493.00,1974.13,18.

In [73]:
exchange_data_df = pd.concat(dfs)

exchange_data_df.shape

(3264, 51)

- Step 4. Clean Up Columns

In [74]:
# check columns
exchange_data_df.columns

Index(['Chinese yuan', 'Euro  (1)', 'Japanese yen', 'U.K. pound  (1)',
       'U.S. dollar', 'Algerian dinar', 'Australian dollar  (1)',
       'Bahrain dinar', 'Botswana pula  (1)', 'Brazilian real',
       'Brunei dollar', 'Canadian dollar', 'Chilean peso', 'Colombian peso',
       'Czech koruna', 'Danish krone', 'Hungarian forint', 'Icelandic krona',
       'Indian rupee', 'Indonesian rupiah', 'Iranian rial',
       'Israeli New Shekel', 'Kazakhstani tenge', 'Korean won',
       'Kuwaiti dinar', 'Libyan dinar  (2)', 'Malaysian ringgit',
       'Mauritian rupee', 'Mexican peso', 'Nepalese rupee',
       'New Zealand dollar  (1)', 'Norwegian krone', 'Omani rial',
       'Pakistani rupee', 'Peruvian sol', 'Philippine peso', 'Polish zloty',
       'Qatari riyal', 'Russian ruble', 'Saudi Arabian riyal',
       'Singapore dollar', 'South African rand', 'Sri Lankan rupee',
       'Swedish krona', 'Swiss franc', 'Thai baht', 'Trinidadian dollar',
       'Tunisian dinar', 'U.A.E. dirham', 'U

- Issues:
    - Need to remove extra characters such as `(1)` , `(2)`, and `.`
    - Need to remove spaces and `concatenate` currency namesß

In [75]:
for col in exchange_data_df.columns:
    pattern = ['(1)', '(2)', '.']
    col = "".join(x.title() for x in col.split() )
    for char in pattern:
        if char in col:
            col = col.replace(char, '')
    print(col)

ChineseYuan
Euro
JapaneseYen
UKPound
USDollar
AlgerianDinar
AustralianDollar
BahrainDinar
BotswanaPula
BrazilianReal
BruneiDollar
CanadianDollar
ChileanPeso
ColombianPeso
CzechKoruna
DanishKrone
HungarianForint
IcelandicKrona
IndianRupee
IndonesianRupiah
IranianRial
IsraeliNewShekel
KazakhstaniTenge
KoreanWon
KuwaitiDinar
LibyanDinar
MalaysianRinggit
MauritianRupee
MexicanPeso
NepaleseRupee
NewZealandDollar
NorwegianKrone
OmaniRial
PakistaniRupee
PeruvianSol
PhilippinePeso
PolishZloty
QatariRiyal
RussianRuble
SaudiArabianRiyal
SingaporeDollar
SouthAfricanRand
SriLankanRupee
SwedishKrona
SwissFranc
ThaiBaht
TrinidadianDollar
TunisianDinar
UAEDirham
UruguayanPeso
BolivarFuerte


### Why compile regex?

- When a regular expression is compiled using `re.compile()`, the pattern is analyzed and transformed into an internal representation that can be used to efficiently match the pattern against strings.

- Compiling a regular expression using `re.compile()` can be useful when the same regular expression pattern will be used multiple times in a program, as it can improve performance by avoiding the need to recompile the pattern each time it is used. Additionally, compiling a regular expression can make the code more readable and easier to maintain, as the compiled regular expression can be assigned to a variable with a descriptive name that makes its purpose clear.

In [77]:
# import re to improve the code 
import re 

def rename_col(col):
    """
    Removes specific patterns from a column name and converts it to title case.

    Args:
        col (str): The column name to be transformed.

    Returns:
        str: The transformed column name with patterns removed and title case applied.
    
    Examples:
        >>> rename_col("col (1)")
        'Col'
        
        >>> rename_col("column (2)")
        'Column'
        
        >>> rename_col("this.is a column")
        'ThisIsAColumn'
    """
    pattern = re.compile(r'\(1\)|\(2\)|\.')
    col = "".join(x.title() for x in col.split() )
    col = pattern.sub('', col)
    return col 

# test
rename_col('U.K. pound  (1)')



'UKPound'

In [78]:
exchange_data_df = exchange_data_df.rename(rename_col, axis=1)
exchange_data_df.head()

Currency,ChineseYuan,Euro,JapaneseYen,UKPound,USDollar,AlgerianDinar,AustralianDollar,BahrainDinar,BotswanaPula,BrazilianReal,BruneiDollar,CanadianDollar,ChileanPeso,ColombianPeso,CzechKoruna,DanishKrone,HungarianForint,IcelandicKrona,IndianRupee,IndonesianRupiah,IranianRial,IsraeliNewShekel,KazakhstaniTenge,KoreanWon,KuwaitiDinar,LibyanDinar,MalaysianRinggit,MauritianRupee,MexicanPeso,NepaleseRupee,NewZealandDollar,NorwegianKrone,OmaniRial,PakistaniRupee,PeruvianSol,PhilippinePeso,PolishZloty,QatariRiyal,RussianRuble,SaudiArabianRiyal,SingaporeDollar,SouthAfricanRand,SriLankanRupee,SwedishKrona,SwissFranc,ThaiBaht,TrinidadianDollar,TunisianDinar,UAEDirham,UruguayanPeso,BolivarFuerte
2010-01-04,6.8281,1.4389,93.05,1.6223,1.0,NaN,0.8970,0.376,0.1506,1.7404,1.4038,1.0378,507.10,2044.23,18.275,5.1717,187.23,124.85,46.51,9330.0,10009.0,3.765,148.46,1167.6,0.2871,1.932367,3.4195,30.3606,12.9226,74.60,NaN,5.7190,0.3845,84.4322,NaN,NaN,2.8465,3.64,NaN,3.75,1.4038,7.315,114.4253,7.1425,1.0336,33.359,6.3574,NaN,3.6725,NaN,2.1446
2010-01-05,6.8278,1.4442,91.75,1.6014,1.0,NaN,0.9133,0.376,0.1514,1.7232,1.3969,1.0372,505.70,2021.21,18.164,5.1527,186.09,124.78,46.20,9308.0,9980.0,3.736,148.46,1156.2,0.2866,1.932367,3.3885,30.3474,12.8241,74.11,0.7344,5.6862,0.3845,84.6402,NaN,NaN,2.8264,3.64,NaN,3.75,1.3969,7.285,114.4404,7.0575,1.0307,33.278,6.3548,NaN,3.6725,NaN,2.1446
2010-01-06,6.8277,1.4350,91.60,1.5984,1.0,NaN,0.9138,0.376,0.1504,1.7219,1.3966,1.0334,504.22,1992.78,18.352,5.1852,187.30,125.29,46.12,9308.0,10000.0,3.740,148.40,1144.3,0.2869,1.932367,3.3875,30.3042,12.7489,73.88,0.7343,5.7059,0.3845,84.7199,NaN,NaN,2.8493,3.64,NaN,3.75,1.3966,7.335,114.3429,NaN,1.0335,33.156,6.3640,NaN,3.6725,NaN,2.1446
2010-01-07,6.8276,1.4304,92.20,1.5912,1.0,NaN,0.9225,0.376,0.1494,1.7329,1.3942,1.0351,497.93,1971.32,18.436,5.2021,187.65,125.54,45.87,9228.0,9981.0,3.727,148.22,1138.2,0.2867,1.932367,3.3705,30.2026,12.7646,73.60,0.7378,5.7313,0.3845,84.7822,NaN,NaN,2.8631,3.64,NaN,3.75,1.3942,7.395,114.3298,7.1125,1.0350,33.160,6.3610,NaN,3.6725,NaN,2.1446
2010-01-08,6.8279,1.4273,93.30,1.6007,1.0,NaN,0.9154,0.376,0.1490,1.7405,1.3982,1.0344,494.54,1969.08,18.436,5.2133,187.91,126.21,45.80,9240.0,NaN,3.718,148.22,1133.2,0.2867,1.932367,3.3785,30.1882,12.7243,73.01,0.7325,5.7237,0.3845,84.7547,NaN,NaN,2.8683,3.64,NaN,3.75,1.3982,7.445,114.3221,7.1225,1.0364,33.130,6.3343,NaN,3.6725,NaN,2.1446


- Step 5. Adjust outlier currencies using `apply()`.
    - outliers = `'Euro', 'UKPound', 'AustralianDollar', 'BotswanaPula', 'NewZealandDollar'`
    - Per `Notes` on IMF page, all rates are expressed in terms of currency units per USD, except:
        - (1) in terms of USD per currency unit
        - (2) in terms of currency units per SDR - Special Drawing Rights, a basket of currency.

In [79]:
# create a callback function and apply it to a sample 'Euro' column
exchange_data_df['Euro'].apply(lambda cur: 1/cur)

2010-01-04    0.694975
2010-01-05    0.692425
2010-01-06    0.696864
2010-01-07    0.699105
2010-01-08    0.700624
                ...   
2023-02-10    0.935454
2023-02-13    0.935804
2023-02-14    0.929454
2023-02-15    0.934579
2023-02-16    0.934579
Name: Euro, Length: 3264, dtype: float64

In [80]:
# apply the logic to all outlier columns and overwrite the df
outliers = ['Euro', 'UKPound', 'AustralianDollar', 'BotswanaPula', 'NewZealandDollar']
exchange_data_df[outliers] = exchange_data_df[outliers].apply(lambda cur: 1/cur)

In [81]:
exchange_data_df.index.rename('Date')
exchange_data_df.columns

Index(['ChineseYuan', 'Euro', 'JapaneseYen', 'UKPound', 'USDollar',
       'AlgerianDinar', 'AustralianDollar', 'BahrainDinar', 'BotswanaPula',
       'BrazilianReal', 'BruneiDollar', 'CanadianDollar', 'ChileanPeso',
       'ColombianPeso', 'CzechKoruna', 'DanishKrone', 'HungarianForint',
       'IcelandicKrona', 'IndianRupee', 'IndonesianRupiah', 'IranianRial',
       'IsraeliNewShekel', 'KazakhstaniTenge', 'KoreanWon', 'KuwaitiDinar',
       'LibyanDinar', 'MalaysianRinggit', 'MauritianRupee', 'MexicanPeso',
       'NepaleseRupee', 'NewZealandDollar', 'NorwegianKrone', 'OmaniRial',
       'PakistaniRupee', 'PeruvianSol', 'PhilippinePeso', 'PolishZloty',
       'QatariRiyal', 'RussianRuble', 'SaudiArabianRiyal', 'SingaporeDollar',
       'SouthAfricanRand', 'SriLankanRupee', 'SwedishKrona', 'SwissFranc',
       'ThaiBaht', 'TrinidadianDollar', 'TunisianDinar', 'UAEDirham',
       'UruguayanPeso', 'BolivarFuerte'],
      dtype='object', name='Currency')

- Step 6. Observe/explore null values.

Handling Null Values

- Common strategies for handling null values in exchange rate data:

    - Remove null values: If the null values represent a small proportion of the dataset or are unlikely to have a significant impact on your analysis, you may choose to simply remove the rows or columns that contain null values using the `dropna()` method in pandas.

    - Replace null values with mean or median: If the null values are relatively few and are missing at random, you can replace them with the mean or median of the corresponding column using the `fillna()` method in pandas.

    - Interpolate missing values: If the null values represent a large proportion of the dataset, you can use interpolation to fill in the missing values. Pandas provides a number of interpolation methods, such as `linear interpolation (interpolate(method='linear'))` or `cubic spline interpolation (interpolate(method='cubic'))`.
    
    - Use a statistical model: If the null values are related to other variables in the dataset, you can use a statistical model to impute missing values. For example, you can use regression analysis or machine learning algorithms to predict missing values based on other features in the dataset.
    
    - Leave null values as is: If the null values are informative (e.g., indicating missing data or unavailable exchange rates) and removing or imputing them would bias your analysis, you may choose to leave them as is and work around them in your analysis.
 

In [82]:
# get a summary of the data
exchange_data_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3264 entries, 2010-01-04 to 2023-02-16
Data columns (total 51 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ChineseYuan        3031 non-null   float64
 1   Euro               3223 non-null   float64
 2   JapaneseYen        3091 non-null   float64
 3   UKPound            3205 non-null   float64
 4   USDollar           3264 non-null   float64
 5   AlgerianDinar      2948 non-null   float64
 6   AustralianDollar   3129 non-null   float64
 7   BahrainDinar       2234 non-null   float64
 8   BotswanaPula       3078 non-null   float64
 9   BrazilianReal      3155 non-null   float64
 10  BruneiDollar       3042 non-null   float64
 11  CanadianDollar     3069 non-null   float64
 12  ChileanPeso        3110 non-null   float64
 13  ColombianPeso      2817 non-null   float64
 14  CzechKoruna        3146 non-null   float64
 15  DanishKrone        3118 non-null   float64
 16  Hungar

In [83]:
# get a count of missing values per column
exchange_data_df.isna().sum()

Currency
ChineseYuan           233
Euro                   41
JapaneseYen           173
UKPound                59
USDollar                0
AlgerianDinar         316
AustralianDollar      135
BahrainDinar         1030
BotswanaPula          186
BrazilianReal         109
BruneiDollar          222
CanadianDollar        195
ChileanPeso           154
ColombianPeso         447
CzechKoruna           118
DanishKrone           146
HungarianForint      1093
IcelandicKrona       1098
IndianRupee           274
IndonesianRupiah     1139
IranianRial          1603
IsraeliNewShekel      234
KazakhstaniTenge     1098
KoreanWon             179
KuwaitiDinar          422
LibyanDinar          1027
MalaysianRinggit      216
MauritianRupee        318
MexicanPeso           235
NepaleseRupee        1052
NewZealandDollar      142
NorwegianKrone        139
OmaniRial             241
PakistaniRupee       1127
PeruvianSol           551
PhilippinePeso        438
PolishZloty           118
QatariRiyal           216
Rus

In [84]:
exchange_data_df.insert(0, 'Date', exchange_data_df.index)

In [85]:
exchange_data_df.columns

Index(['Date', 'ChineseYuan', 'Euro', 'JapaneseYen', 'UKPound', 'USDollar',
       'AlgerianDinar', 'AustralianDollar', 'BahrainDinar', 'BotswanaPula',
       'BrazilianReal', 'BruneiDollar', 'CanadianDollar', 'ChileanPeso',
       'ColombianPeso', 'CzechKoruna', 'DanishKrone', 'HungarianForint',
       'IcelandicKrona', 'IndianRupee', 'IndonesianRupiah', 'IranianRial',
       'IsraeliNewShekel', 'KazakhstaniTenge', 'KoreanWon', 'KuwaitiDinar',
       'LibyanDinar', 'MalaysianRinggit', 'MauritianRupee', 'MexicanPeso',
       'NepaleseRupee', 'NewZealandDollar', 'NorwegianKrone', 'OmaniRial',
       'PakistaniRupee', 'PeruvianSol', 'PhilippinePeso', 'PolishZloty',
       'QatariRiyal', 'RussianRuble', 'SaudiArabianRiyal', 'SingaporeDollar',
       'SouthAfricanRand', 'SriLankanRupee', 'SwedishKrona', 'SwissFranc',
       'ThaiBaht', 'TrinidadianDollar', 'TunisianDinar', 'UAEDirham',
       'UruguayanPeso', 'BolivarFuerte'],
      dtype='object', name='Currency')

- Step 7. Save the final data to a csv file. 

In [86]:
exchange_data_df.to_csv('../data/IMF_DailyExchangeRates_2010To2023_AllCurrencies.csv', index=False)